In [ ]:
import laser_lib
import numpy as np
import math
import time
import cv2
from skimage.color import rgb2lab, lab2rgb

In [ ]:
queue = laser_lib.DacQueue()

In [ ]:
def signer(f, T, phase=0.0):
    '''
    Produces spaced out points from 0 to 1 whose distance between 
    consecutive points is sinusoidal 
    Input:  f - frequency 
            T - number of samples
            phase - phase shift (0, 1)
    Returns: x - positions on the line
             dx - the derivative of the function of x
    '''
    # Convert to radians
    f = f*2*np.pi
    # Generate the baseline 
    t = np.linspace(0.0+phase, 1.0+phase, T)
    # Find points 
    x = (f*t - np.cos(f*t) + 1)/f
    # Calculate derivative
    dx = (np.sin(f*t) + 1)/2
    
    return x, dx

def triangle(theta, alpha, sincos='sin'):
    '''
    Returns a wave between a sine wave and triangle wave depending on alpha
    alpha = 0 - sine wave
    alpha = 1 - triangle wave
    '''
    if(sincos == 'sin'):
        ps = 0
    elif(sincos == 'cos'):
        ps = np.pi/2
    return alpha*2*np.arcsin(np.sin(theta + ps))/np.pi + (1-alpha)*np.sin(theta + ps)
    
def ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=1000, phase_s=0, sintriang=0, colors=[(1,1,1)], T=500):
    '''
    Generates a pattern that is a function of ellipses 
    with parameters that can be interpolated between
    '''
    theta_norm, theta_dx = signer(f=f_s, phase=phase_s, T=T)
    theta = theta_norm*2*np.pi
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = scale_x*triangle(f_x*theta, sintriang, 'sin')
    arr_pos[:,1] = scale_y*triangle(f_y*theta, sintriang, 'cos')
    
    # Color the pattern
    arr_col = np.ones((T, 3), np.float32)
    if(len(colors) == 1):
        # All one color
        arr_col[:, :] = colors[0][np.newaxis, :]
    if(len(colors) == 2):
        # Color based on speed of laser
        arr_col[theta_dx > 0.1, :] = colors[0]
        arr_col[theta_dx <= 0.1, :] = colors[1]
    
    return arr_pos, arr_col


### Beam pairs flashing back and forth

In [ ]:
num_beams = 2
num_positions = 2
all_positions = np.linspace(-1.0, 1.0, num_beams*num_positions)
all_colors = np.array([[1,0,1],[0.7,0.7,0.7]])

T = 1000
diver = 5
while(True):
    for pos_idx in range(num_positions):
        positions = all_positions[pos_idx::num_beams]
        print(positions)
        # Clear the arrays
        arr_pos = np.zeros((T, 2))
        arr_col = np.zeros((T, 3))
        # Populate them
        arr_pos[::T//5]
        
    
    

In [ ]:
T = 1000
theta = np.linspace(0, 2*np.pi, T)
freq = 10
arr_pos_l = np.zeros((T, 2))
arr_pos_r = np.zeros((T, 2))
arr_col_l = np.zeros((T, 3))
arr_col_r = np.zeros((T, 3))
scaler = 0.75
arr_pos[:, 0] = np.cos(freq*theta)
arr_pos_l[:, 0] = np.cos(freq*theta)*scaler + (1-scaler)
arr_pos_r[:, 0] = np.cos(freq*theta)*scaler - (1-scaler)
arr_col_l[arr_pos[:,0] >  0.99, :] = np.array([1,1,1])/6
arr_col_r[arr_pos[:,0] < -0.99, :] = np.array([1,1,1])/6
arr_col_l[arr_pos[:,0] < -0.99, :] = np.array([1,0,1])
arr_col_r[arr_pos[:,0] >  0.99, :] = np.array([1,0,1])

stay = 10
while(True):
    for i in range(stay):
        queue.submit(arr_pos_l, arr_col_l)
        
    for i in range(stay):
        queue.submit(arr_pos_r, arr_col_r)

### Concentric, Rotating, Discrete Beam Circles

In [ ]:
def rotate_pos(arr_pos, ang_rot):
    '''
    Rotates all points in arr_pos by ang_rot radians
    '''
    c, s = np.cos(ang_rot), np.sin(ang_rot)
    R = np.array(((c, -s), (s, c)))
    return np.matmul(arr_pos, R)

ang_rot = 0
T = 600
num_beams = 6
num_cols = 3
points_per_beam = T//num_beams
trip_col = np.array([[1,0,1],[0,0.5,1],[0,0,1]])
trip_col = np.tile(trip_col, (num_beams//num_cols, 1))
trip_col = np.repeat(trip_col, points_per_beam, axis=0)

while(True):
    ang_rot += 0.1
    arr_pos, arr_col = ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=num_beams, phase_s=0, sintriang=0, colors=[(0,0,0),(1,1,1)], T=T)
    queue.submit(rotate_pos(arr_pos, ang_rot), trip_col*arr_col)
    queue.submit(0.75*rotate_pos(arr_pos, -ang_rot), trip_col*arr_col)
    queue.submit(0.50*rotate_pos(arr_pos,  ang_rot), trip_col*arr_col)
    

In [ ]:
# ARC THAT EXPANDS and collapes formulas
T = 1000
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

num_steps = 100
#Arc expands in both directions from Top of circle (pi/2)
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)

#Arc disappears in both directions from Top of circle (pi/2)    
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)

#Arc disappears in both directions from 45deg of circle (pi/4)     
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/4, theta[:, np.newaxis] < np.pi/4 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)
    
#Arc disappears in both directions from 135deg of circle (3pi/4)     
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + 3*np.pi/4, theta[:, np.newaxis] < 3*np.pi/4 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)
    
#Arc disappears from right and left side of circle (0 and pi)     
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] > np.pi*i/num_steps, theta[:, np.newaxis] < np.pi - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)
    


In [ ]:
# ARC THAT EXPANDS and collapses in order
T = 1000
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

#num steps = 30fps * 6 seconds + 1 beat, 1 beat = 3/8s = 90/8 frams= 11.25 or so frames, quick quick slow = 11, 11, 23 = 45 = 11.25*4
num_steps = 191
#Arc expands in both directions from Top of circle (pi/2)
for i in range(num_steps):
    arr_col_out = arr_col*(theta[:, np.newaxis] = np.pi/2)
    i-= 5
    arr_col_out = arr_col*np.locgical_and(theta[:, np.newaxis] = np.pi/4, theta[:, np.newaxis] = 3*np.pi/4)
    i-= 5
    arr_col_out = arr_col*np.locgical_and(theta[:, np.newaxis] = 0, theta[:, np.newaxis] = np.pi)
    i-= 1
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
    i -= 90
    arr_col_out1 = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
    arr_col_out2 = arr_col*(theta[:, np.newaxis] = np.pi/2)
    arr_col_out = np.logical_and(arr_col_out1, arr_col_out2)
    i -= 17 
    arr_col_out1 = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/4, theta[:, np.newaxis] < np.pi/4 - np.pi*i/num_steps)
    arr_col_out2 = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + 3*np.pi/4, theta[:, np.newaxis] < 3*np.pi/4 - np.pi*i/num_steps)
    arr_col_out3 = arr_col*np.locgical_and(theta[:, np.newaxis] = np.pi/4, theta[:, np.newaxis] = 3*np.pi/4)
    arr_col_out = np.logical_and(arr_col_out1, arr_col_out2, arr_col_out3)
    i -= 17
    arr_col_out1 = arr_col*np.logical_and(theta[:, np.newaxis] > np.pi*i/num_steps, theta[:, np.newaxis] < np.pi - np.pi*i/num_steps)
    arr_col_out2 = arr_col*np.locgical_and(theta[:, np.newaxis] = 0, theta[:, np.newaxis] = np.pi)
    arr_col_out = np.logical_and(arr_col_out1, arr_col_out2)
    queue.submit(arr_pos, arr_col_out)
#big concern is I don't think the individual rays will be gone. I think i will have them persist throughout, 
#but I need to reset arr.col.out to 0 for those points only. Write additional code to say such?


In [ ]:
# Frames -> Patterns -> Sequences

In [ ]:
# Opening Beats 0-8 seconds, switching rays
T = 1000
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

#not eliminating lasers right now. Just have timing down, need to set up way to clear arr_col_out after each pass
num_steps = 330
for i in range(num_steps):
    i-= 11 #.375s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 11 #.75s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #1.125s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #1.5s (correction) (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #1.875s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 11 #2.25s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #2.625s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #3s (correction) (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #3.375s - First Background beat - (a) (1)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6, theta[:, np.newaxis] = np.pi/2)
    i-= 11 #3.75s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 6  #3.9375s (2)
    arr_col_out = arr_col*np.locgical_and(theta[:, np.newaxis] = np.pi/4, theta[:, np.newaxis] = 3*np.pi/4)
    i-= 5  #4.125 (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #4.5 (correction) (b) (3)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3, theta[:, np.newaxis] = 0, theta[:, np.newaxis] = np.pi)    
    i-= 11 #4.875s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 11 #5.25s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #5.625s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #6s (correction) (b) (1)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3, theta[:, np.newaxis] = np.pi/2)
    i-= 5  #6.1875s (1) (2)
    arr_col_out = arr_col*np.locgical_and(theta[:, np.newaxis] = np.pi/4, theta[:, np.newaxis] = 3*np.pi/4)
    i-= 6  #6.375s (a) (1) (2) (3)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6, theta[:, np.newaxis] = 0, theta[:, np.newaxis] = np.pi)
    i-= 11 #6.75s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #7.125s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #7.5s (correction) (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #7.875s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 11 #8.25s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 5  #8.4375s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 6  #8.625s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 6  #8.8125s (correction) (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 6  #9s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 5 #9.1875 (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 6 #9.375 (ripple)
    


In [ ]:
# ARC THAT EXPANDS
T = 500
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

num_steps = 100
while(True):
    for i in range(num_steps-30):
        arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
        i -= 20
        arr_col_out = arr_col_out*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
        queue.submit(arr_pos, arr_col_out)

    for i in range(num_steps-30):
        arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
        i -= 20
        arr_col_out = arr_col_out*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
        queue.submit(arr_pos, arr_col_out)


In [ ]:
for i in range(10):
    print(i)
    i += 20
    print(i)

In [ ]:
thet = 2
x = 0.5

In [ ]:
theta + pi/ 2 > x & theta + pi/ 2 < x

In [ ]:
x = np.array([True, True, False, False])
y = np.array([True, False, True, False])
print(np.logical_or(x, y))
print(np.logical_and(x, y))

In [ ]:
print(queue.dac_rate)

In [ ]:
# Time of the entire pattern
Ts = 6.0
# Points per frame
T = 1000
# Time per frame
Tf = T/queue.dac_rate
# Time for each segment
T_seg = 1.0

num_frames = int(Ts/Tf)
print(num_frames)
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_col = np.ones((T, 3))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)
for frame_idx in range(num_frames):
    queue.submit(arr_pos, arr_col)
    

    

In [ ]:
Tf